# Fusion the datasets
The aim of this notebook is to gather all the produced datasets(Only those produced by filtering are implemented here), and then split them into test and train.

# Variables

In [0]:
import os
import csv
#TO Implement   
    #mounting drive path, this is optional
drive_path = ''
# root path as defined in install.ipynb 
data_root = ''

save_root = os.path.join(data_root, 'results/VGQA')
save_qas_file = os.path.join(save_root,'QAs.json')
filtred_GQA = os.path.join(save_root, 'GQA_filtered.csv')
VG_GQA = os.path.join(save_root, 'VG_GQA.csv')
VQA_train_file = os.path.join(data_root, 'results/VQA/final_train.csv')
VQA_val_file = os.path.join(data_root, 'results/VQA/final_val.csv')

final_val_file = os.path.join(data_root, 'results/final_val.csv')
final_train_file = os.path.join(data_root, 'results/final_train.csv')

TRAIN_RATIO = 0.6 #Train dataset ration between 0 and 1 

## Mounting Drive (Don't do it if you don't use drive)

In [0]:
from google.colab import drive
#mounting drive path, this is optional
drive_path = ''
drive.mount(drive_path)

# Fusion

In [0]:

def fusion(list1, list2): 
  images1 = set([x[0] for x in list1])
  first_values = [x for x in list2 if x[0] in images1]
  second_values = [x for x in list2 if not x[0] in images1 ]
  return (list1 + first_values, second_values)

def read_csv(file) : 
  with open(file, 'r') as f : 
    reader = csv.reader(f)
    data = [x for x in reader]
  return data

def write_csv(save_file, data): 
  with open(save_file, 'w') as f : 
    writer = csv.writer(f)
    for d in data :   
      writer.writerow(d)

In [0]:
def get_VG(file_path): 
  with open(file_path, 'r') as f:
    data = json.load(f)
  return [[y['image_id'],x['question'],x['answer']] for y in data for x in y['QAs']]

VG = get_VG(save_qas_file)
GQA = read_csv(filtred_GQA)
VG2 = [[str(x[0]), str(x[1]), str(x[2])] for x in VG]
GQA  = [[str(x[0]), str(x[1]), str(x[2])] for x in GQA]
data = VG2 + GQA
data = [[y.replace('\x00', '') for y in x] for x in data]

with open(VG_GQA, 'w') as f :
  writer = csv.writer(f)    
  for x in data : 
    writer.writerow(x)

In [0]:

vgqa = read_csv(VG_GQA)
train_vqa = read_csv(VQA_train_file)
val_vqa = read_csv(VQA_val_file)
new_train, new_vgqa = fusion(train_vqa, vgqa)
new_val , new_new_vgqa = fusion (val_vqa, new_vgqa)

# Split into train and val

In [0]:
total = len(new_train) + len(new_val) + len(new_new_vgqa)
train_split_ratio = (TRAIN_RATIO* total - len(new_train))/ len(new_new_vgqa)
train_length = int(train_split_ratio * len(new_new_vgqa))
final_train = new_train + new_new_vgqa[:train_length]
final_val = list(new_val) + list(new_new_vgqa)[train_length:]

In [0]:
write_csv(final_train_file, final_train)
write_csv(final_val_file, final_val)